# 06. Ch 04. Sequence-to-Sequence - 06. Attention

In [ ]:
<Previous Work>
인코더의 히든 스테이트가 디코더의 이니셜 히든 스테이트가 될텐데 그때 (capacity에서)정보의 한계가 생긴다.
m이 커지면 커질수록 모든 정보를 다 압축하지 못하는 경우가 있다.
혹은 인코더의 성능이 떨어질 수도 있고.
이러한 경우에도 정보가 덜 담겨 있을테고.
정보가 충분하지 않다고 하면 정보를 바탕으로 문장을 생성하는데 어려움을 겪을 수밖에 없다
그때 유효하게 동작하는 게 어텐션이다.

<어텐션이란?>
-Differentiable(미분 가능한) Key-Value Function
d[query] => value 리턴(딕셔너리 내에 키를 비교해)
기존의 파이썬에서 사용하는 딕셔너리 키밸류 펑션은 완벽하게 쿼리가 키와 일치할 경우에만 밸류를 가져온다.
일치하는 경우가 없으면 None을 뱉어내거나 에러가 나기도 함.

-(미분가능한 키밸류 함수는)기존의 키밸류 함수와 달리, 쿼리와 키의 유사도에 따라 밸류를 반환
얘는 30퍼센트 정도만 비슷하니까 30퍼센트 정도로만 밸류를 리턴하자 이런 느낌.
-디코더 RNN(LSTM)의 히든 스테이트의 한계로 인해 부족한 정보를 직접 직접 인코더에 조회해 예측에 필요한 정보를 얻어오는 과정
-정보를 잘 얻어오기 위해 쿼리를 잘 만들어내는 과정을 학습
정보를 가져오는 과정 자체가 어텐션. 그 과정에서 쿼리를 만들어내는 과정을 학습해야 한다.

<Attention in Seq2seq>
-Query: 현재 타임 스텝의 디코더 아웃풋
-Keys: 각 타임 스탭별 인코더 아웃풋
-Values: 각 타임스텝별 인코더 아웃풋

<Intuitive(직관적인) Explanations>
디코더의 쿼리를 인코더의 키와 잘 맞아떨어지게끔 바꿔줘야.
그래야 적절한 유사도가 뜨니까.
이를 위해 리니어 레이어를 통과시켜주는데(Linear Transform)
이 리니어 트랜스폼이 잘 학습돼야 좋은 성능이 나온다.
 
<Linear Transformation>
리니어 트랜스폼을 배운다고 하는 것은
마음의 상태(state)를 잘 반영하면서
좋은 검색 결과를 이끌어내는 쿼리를 얻기 위함.

어텐션은 쿼리를 잘 만드는 과정을 배우는 것이다.

<Before we start,>
-Batch Matrix Multiplication(BMM) 배치 행렬곱
3차원의 배치 행렬곱은 배치 사이즈가 모두 동일해야 한다.
파이토치에서의 사용은
z = torch.bmm(x, y)
어텐션에서 사용하게 된다.
<어텐션>
현재 타임 스텝의 디코더의 히든 스테이트와
인코더의 히든 스테이트를 활용해 구한 어텐션 웨이트로 컨텐스트 벡터를 구하고
컨텍스트 벡터와 현재 타임 스텝의 히든 스테이트를 concat해
소프트맥스 레이어에 넣어준다.

<평가>
어텐션이 들어가지 않은 모델 같은 경우에는 문장의 길이가 길어질수록 성능이 떨어짐
어텐션이 들어간 경우에는 길이가 길어져도 성능이 유지된다.
긴 문장에 대해서도 충분히 잘 대처할 수 있다.

<요약>
-어텐션은 미분 가능한 키밸류 함수이다.
어텐션 함수의 입력은 쿼리, 키, 밸류

-정보를 잘 얻기 위한 쿼리를 변환하는 방법을 배우는 과정
디코더의 타임스텝마다 모자란 정보가 있을 수 있다. 알쏭달쏭한 정보.
그때 필요한 정보를 얻어오기 위해 쿼리를 잘 변환해야 한다.
(리니어 트랜스포메이션을 통해 쿼리를 잘 변환 가능)

-어텐션을 통해 RNN의 히든 스테이트의 한계를 극복 가능
LSTM을 쓰더라도 컨텍스트 벡터에 모든 정보를 담기에는 한계가 있음
더 긴 길이의 입력/출력에도 대처할 수 있게 됨.
그 결과 엄청난 성능 향상이 있었다.
더 긴 문장을 디코더가 잘 생성할 수 있게 됨.

어텐션에 더해 시퀀스투시퀀스를 보완하는 방법을 살펴보겠다.

# 07. Ch 04. Sequence-to-Sequence - 07. Masking

In [ ]:
Practical Issue: Masking on Attention(실제 논문에서도 잘 다루지 않는 실용적 이슈)

<Motivation>
우리는 문장을 미니배치로 묶어서 학습을 한다.
추론할 때도 패러렐하게 오퍼레이션을 진행한다.

미니 배치 내 문장들이 길이가 같지 않다.
문장 구성이 다름. 짧은 문장, 긴 문장.
긴 문장에 의해 미니배치 길이가 결정.
짧은 문장들에는 패딩 <PAD>으로 채워지게 된다.

이 시간에 배우는 거 처리 안 해줘도 큰 성능 저하 없이 쓸 수 있다.
하지만 문제가 생긴다.
어쨌든 처리를 해줘야 한다. 상용화 수준을 생각한다면.

<Assign Attention Weights to <PAD>>
리니어 트랜스폼을 통해 패드에도 어텐션 연산이 들어가게 된다.
키와 쿼리가 곱해지게 되는데 웨이트가 0이 나오기 힘들다.
그러면 무조건 어떤 웨이트가 들어가는 수밖에 없다.
웨이트가 들어가면 밸류에서 웨이트가 곱해져서 패드에 해당하는 가비지 정보가
웨이티드 썸으로 들어가게 되고 컨텍스트 벡터에는 쓸데없는 패드라는 정보가 들어가게 된다.

이를 막기 위한 안전장치를 만들어야 한다.
머신러닝과 관련된 것은 아니고 엔지니어링과 관련.
이렇게 안 해도 잘 되지만
안전장치가 없으면
추론을 할 때
미니배치 구성에 따라(m의 길이에 따라) 패드가 바뀐다.
패드의 개수에 따라서 유사도에 어텐션이 들쑥날쑥하게 들어가게 되니까
쓸데없는 어텐션의 결과들이 막 바뀌어서 들어간다는 것.
어제는 I love you였는데 패드 6개
오늘은 I love you였는데 패드 3개
그러면 번역 결과가 바뀐다.
대충은 맞음. 나는 사랑해 너를, 나는 너를 사랑한다 식으로 마이너하게 바뀌지만.
안전장치가 없으면 그 상태로 학습이 됐고 그때 어떻게 동작하는지 뉴럴 네트워크가 배웠기 때문에
패드 정보가 들어가면 무시한다 이런 식으로 어느 정도 대처가 된다.
하지만 사용자 입장에서 일정하지 않은 결과값이 나오게 되므로 안전장치를 도입해야 함.

<솔루션>
마스크를 하나 만들어서
쿼리와 키를 곱한 결과가 나왔을 때 마스크의 형태대로 마이너스 무한대로 바꿔치는 것.
그렇게 되면 소프트맥스 결과(어텐션 웨이트)에는 패드 자리가 0이 된다.

<요약>
-미니배치 내의 문장 구성에 따라 패드가 동적으로 생성됨
패드의 히든 스테이트에는 어텐션 웨이트가 할당되면 안 됨.
-따라서 키와 쿼리의 닷 프로덕트 이후에(소프트맥스 이전에) 마스킹을 통해 패드 위치의 값을 음의 무한대로 변경
소프트맥스 결과 패드에는 0이 할당됨.
-이 기법은 이후 트랜스포머에서도 유용하게 쓰일 것.

# TODO: 08. Ch 04. Sequence-to-Sequence - 08. Input Feeding